# TASK 1 : Install Dependencies & Run a SparkSession


In [ ]:
#install pyspark
!pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 39.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=22df9606922a2b14c2cfb98e18d4711de4bdc193e3bd2242c3150961d77c23d3
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession

#create spark session
spark = SparkSession.builder.appName("AppName").getOrCreate()

# TASK 2 : Clone & Explore dataset

In [ ]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [ ]:
#check the presence of dataset
!ls admission_dataset/

Admission_Predict_Ver1.1.csv


In [ ]:
#create a spark dataframe. spark df are immutable.
#distributed collections of data organized into named columns
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv', header=True, inferSchema=True)


In [ ]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [ ]:
#get the no.of rows & columns
print(df.count(), len(df.columns))


500 9


In [ ]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#get the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

# TASK 3 : Data Cleaning

In [ ]:
#drop the unnecessary column
df = df.drop('Serial No')

In [ ]:
#display the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [ ]:
#check for null values. Filter on the null values and count it.
for i in df.columns:
  print(i+":", df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


# TASK 4 : Correlation Analysis & Feature Selection

In [ ]:
# correlation analysis
# get the correlation among the set of input and the output columns
for col in df.columns:
  print('Correlation to chance of admit col for {} is {}'.format(col,df.stat.corr('Chance of Admit',col)))

Correlation to chance of admit col for GRE Score is 0.8103506354632598
Correlation to chance of admit col for TOEFL Score is 0.7922276143050823
Correlation to chance of admit col for University Rating is 0.6901323687886892
Correlation to chance of admit col for SOP is 0.6841365241316723
Correlation to chance of admit col for LOR is 0.6453645135280112
Correlation to chance of admit col for CGPA is 0.882412574904574
Correlation to chance of admit col for Research is 0.5458710294711379
Correlation to chance of admit col for Chance of Admit is 1.0


In [ ]:
# feature selection
#'GRE Score','TOEFL Score','CGPA' are the highly correlated columns.
# Vector Assembler use as A feature transformer that merges multiple columns into a vector column.
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score','CGPA'],outputCol='features')

In [ ]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

# TASK 5 : Build the Linear Regression Model

In [ ]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features','Chance of Admit')

In [ ]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#split the dataset into training and testing set using randomSplit() function
train, test = final_data.randomSplit([0.7, 0.3])

In [ ]:
#build & train the model
models = LinearRegression(featuresCol='features', labelCol='Chance of Admit')
model = models.fit(train)

In [ ]:
#get coefficients & intercept
print("coefficients", model.coefficients)
print('intercept', model.intercept)

coefficients [0.0029917153377179255,0.003023218431318594,0.13094085447106413]
intercept -1.6710997274176267


In [ ]:
#get summary of the model
summary = model.summary

In [ ]:
#print the rmse & r2 score - predicted v/s actual value difference
print('RMSE',summary.rootMeanSquaredError)
print('r2',summary.r2)  #how close data is to regression line

RMSE 0.060257755024189286
r2 0.8143068735743533


# TASK 6 : Evaluate & Save the Model

In [ ]:
#transform on the test data
predictions = model.transform(test)

In [ ]:
#display the predictions
predictions.show(20)

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|[290.0,100.0,7.56]|           0.47| 0.4887324234536756|
|  [293.0,97.0,7.8]|           0.64| 0.5200637192459292|
| [295.0,99.0,7.57]|           0.37| 0.5019771902556573|
| [296.0,99.0,8.03]|           0.61| 0.5652016986500648|
|[296.0,101.0,7.68]|            0.6|   0.52541883644783|
|  [298.0,98.0,7.5]|           0.44|0.49876325802451826|
|[298.0,100.0,7.95]|           0.58| 0.5637330793991346|
|[299.0,100.0,7.88]|           0.51| 0.5575589349238779|
|  [300.0,97.0,8.1]|           0.65| 0.5802879829512737|
|  [300.0,99.0,6.8]|           0.36| 0.4161113090015276|
|[300.0,102.0,8.17]|           0.63| 0.6045699349208415|
|[300.0,104.0,8.16]|           0.71| 0.6093069632387682|
| [301.0,97.0,7.88]|           0.44| 0.5544727103053575|
| [301.0,99.0,8.22]|           0.64| 0.6050390376881569|
| [301.0,99.0,8.45]|           

In [ ]:
# Use RegressionEvaluator for model evaluation
from pyspark.ml import evaluation
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit',metricName='r2')
print('r2 on the test data', evaluator.evaluate(predictions))

r2 on the test data 0.7786100671218159


In [ ]:
#save the model
model.save("model")

In [ ]:
#load the model
from pyspark.ml.regression import LinearRegressionModel
model = LinearRegressionModel.load('model')